## Setup

In [7]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.base import BaseEstimator, RegressorMixin
import json

import datetime
import calendar
import time
from sklearn import tree
import matplotlib.pyplot as pls

## Loading flow with rain

In [8]:
PROJECT_FOLDER = './data/'

with open(PROJECT_FOLDER + 'project.json', 'r') as file:
    project = json.load(file)
print(json.dumps(project, indent=4))

def load_series(fname, name):
    path = PROJECT_FOLDER + fname + '.csv'
    xs = pd.read_csv(path, parse_dates=['time'])
    xs = xs.set_index('time')[name].fillna(0)
    xs = xs.resample('5T').pad()
    xs = xs.rename(fname)
    return xs
    

flow1_raw = load_series('flow1', 'flow')
flow1_edited = load_series('flow1_edited', 'flow_edited')

flow2_raw = load_series('flow2', 'flow')
flow2_edited = load_series('flow2_edited', 'flow_edited')

flow3_raw = load_series('flow3', 'flow')
flow3_edited = load_series('flow3_edited', 'flow_edited')

flow4_raw = load_series('flow4', 'flow')
flow4_edited = load_series('flow4_edited', 'flow_edited')

flow5_raw = load_series('flow5', 'flow')
flow5_edited = load_series('flow5_edited', 'flow_edited')

rainfall = load_series('rainfall1', 'rainfall')
df = pd.concat(
    [flow1_raw, 
     flow1_edited,
     flow2_raw, 
     flow2_edited,
     flow3_raw, 
     flow3_edited,
     flow4_raw, 
     flow4_edited,
     flow5_raw, 
     flow5_edited,
     rainfall
    ], axis=1)
data_frame = df['2016-01-01':]
data_frame.head()

{
    "rainfalls": [
        "rainfall"
    ], 
    "name": "thorium-large", 
    "start-date": "2015-06-02", 
    "flows": [
        "flow1", 
        "flow2", 
        "flow3", 
        "flow4", 
        "flow5"
    ], 
    "split-date": "2017-01-01", 
    "end-date": "2017-10-01"
}


,flow1,flow1_edited,flow2,flow2_edited,flow3,flow3_edited,flow4,flow4_edited,flow5,flow5_edited,rainfall1
time,,,,,,,,,,,
2016-01-01 00:00:00,93.889999,93.889999,420.320007,420.320007,252.089996,252.089996,225.460007,225.460007,146.110001,146.110001,0.254
2016-01-01 00:05:00,93.480003,93.480003,416.660004,416.660004,251.850006,251.850006,225.470001,225.470001,145.320007,145.320007,0.254
2016-01-01 00:10:00,92.830002,92.830002,416.670013,416.670013,260.440002,260.440002,225.190002,225.190002,144.899994,144.899994,0.254
2016-01-01 00:15:00,92.199997,92.199997,415.059998,415.059998,255.789993,255.789993,225.229996,225.229996,140.699997,140.699997,0.254
2016-01-01 00:20:00,91.959999,91.959999,414.279999,414.279999,255.550003,255.550003,225.699997,225.699997,140.259995,140.259995,0.254


## Fitting a Model

In [ ]:
data = pd.read_csv('data/air_passengers.csv', parse_dates=['date'])
plt.figure(figsize=(9,3))
plt.plot(data.date, data.y, '.-', c='gray')

t_diff_median = data.date.diff().median()
print 'Average time step: {!s}'.format(t_diff_median)

In [ ]:
def fit_model(t, X, y, distance, test_size=0.2, log_transform=False, estimator=None, plot=False):
    
    # apply forecast distance
    X = X.shift(distance)
    
    # ignore missing value rows
    non_null = X.dropna(how='any').index.values

    # create holdout set
    if isinstance(test_size, float):
        test_size = int(test_size * len(t))
    train = non_null[:-test_size]
    test  = non_null[-test_size:]

    # standardize
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    centers = X.loc[train, numeric_cols].mean(axis=0)
    scales = np.maximum(1e-15, X.loc[train, numeric_cols].std(axis=0))
    X[numeric_cols] = (X[numeric_cols] - centers)/scales
    
    # fit model
    if estimator is None:
        estimator = ElasticNetCV(n_alphas=100, l1_ratio=0.9, cv=5, random_state=123)
    if log_transform:
        estimator = MultiplicativeEstimator(estimator)
    estimator.fit(X.loc[train, :], y[train])
    test_score = estimator.score(X.loc[test, :], y[test])

    # plot the fit
    if plot:
        plt.figure(figsize=(9,3))
        plt.plot(t, y, '.', c='gray', alpha=0.7)
        plt.plot(t[train], estimator.predict(X.loc[train, :]), 'b-', lw=2, alpha=0.7)
        plt.plot(t[test], estimator.predict(X.loc[test, :]),  'r-', lw=2, alpha=0.7)
        plt.annotate('Test Score: {}'.format(test_score), 
                     xy=(0, 1), xytext=(12, -12), va='top', 
                     xycoords='axes fraction', textcoords='offset points')
        plt.show()
    return estimator, test_score


def feature_importance_summary(model):
    if hasattr(model, 'coef_'):
        coef = pd.Series(model.coef_, index=X.columns, name='Importance')
        importances = coef.abs()
        importances.sort_values(ascending=False, inplace=True)
        importances = importances[importances > 1e-15]
        display(importances.to_frame().style.bar())
    else:
        raise NotImplemented('Only linear models supported at the momemnt.')
        

class MultiplicativeEstimator(BaseEstimator, RegressorMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator
        
    def fit(self, X, y, *args, **kwargs):
        self.estimator.fit(X, np.log(y), *args, **kwargs)
        return self
    
    def predict(self, X):
        return np.exp(self.estimator.predict(X))
    
    def __getattr__(self, name):
        return getattr(self.estimator, name)

In [ ]:
# read the data
data = pd.read_csv('data/air_passengers.csv', parse_dates=['date'])
t = data['date']
y = data['y']

# create features in new dataframe
X = pd.DataFrame()
X['lag 0'] = y.shift(0)
X['lag 1'] = y.shift(1)
X['lag 2'] = y.shift(2)
X['lag 3'] = y.shift(3)
X['lag 4'] = y.shift(4)
X['lag 5'] = y.shift(5)
X['lag 6'] = y.shift(6)
X['lag 7'] = y.shift(7)
X['lag 8'] = y.shift(8)
X['lag 9'] = y.shift(9)
X['lag 10'] = y.shift(10)
X['lag 11'] = y.shift(11)
X['lag 12'] = y.shift(12)

# fit
model, score = fit_model(t, X, y, plot=True, distance=12)

# display importances
feature_importance_summary(model)

## Number of Lags

In [ ]:
# read the data
data = pd.read_csv('data/air_passengers.csv', parse_dates=['date'])
t = data['date']
y = data['y']
X = pd.DataFrame()

lag_counts = range(0, 20)
scores = []

for i in lag_counts:
    X['lag {}'.format(i)] = y.shift(i)
    model, score = fit_model(t, X, y, distance=12)
    scores.append(score)
    
plt.figure()
plt.plot(lag_counts, scores, lw=2)
plt.xlabel('# of Lags')
plt.ylabel('Test Score')


## More Types of Features

In [ ]:
# read the data
data = pd.read_csv('data/air_passengers.csv', parse_dates=['date'])
t = data['date']
y = data['y']

# create features in new dataframe
X = pd.DataFrame()
X['lag 0'] = y.shift(0)
X['lag 1'] = y.shift(1)
X['lag 2'] = y.shift(2)
X['lag 3'] = y.shift(3)
X['lag 4'] = y.shift(4)
X['lag 5'] = y.shift(5)
X['lag 6'] = y.shift(6)
X['lag 7'] = y.shift(7)
X['lag 8'] = y.shift(8)
X['lag 9'] = y.shift(9)
X['lag 10'] = y.shift(10)
X['lag 11'] = y.shift(11)
X['lag 12'] = y.shift(12)
X['t'] = (t - t.min())/(t.max() - t.min())
X['rolling mean 3'] = y.rolling(3).mean()
X['rolling mean 6'] = y.rolling(6).mean()
X['rolling mean 12'] = y.rolling(12).mean()
X['rolling std 12'] = y.rolling(12).std()
X['rolling min 12'] = y.rolling(12).min()
X['rolling max 12'] = y.rolling(12).max()
X['rolling median 12'] = y.rolling(12).median()

model, score = fit_model(t, X, y, plot=True, distance=12)

feature_importance_summary(model)

# Time Learning Curve

In [ ]:
row_counts = range(X.shape[0] / 2, X.shape[0] + 1)
scores = []

for n in row_counts:
    recent = X.index[-n:]
    model, score = fit_model(t.loc[recent], X.loc[recent, :], y.loc[recent], distance=12)
    scores.append(score)

plt.figure()
plt.plot(100.0*np.array(row_counts)/X.shape[0], scores, lw=2)
plt.axhline(max(scores), linestyle='--', color='gray')
plt.xlabel('% recent rows')
plt.ylabel('Test Score')

plt.figure(figsize=(9,3))
plt.plot(t, y, '-', lw=2)
plt.axvline(t.iloc[-np.argmax(row_counts)], linestyle='--', color='gray')

## Forecast Distance

In [ ]:
distances = range(0, 36)
scores = []

for i in distances:
    model, score = fit_model(t, X, y, distance=i)
    scores.append(score)

plt.figure()
plt.plot(distances, scores, lw=2)
plt.xlabel('Forecast Distance')
plt.ylabel('Test Score')

## Log Transform / Differencing

In [ ]:
# read the data
data = pd.read_csv('data/air_passengers.csv', parse_dates=['date'])
t = data['date']
y = data['y']

# normal
plt.figure(figsize=(9,3))
plt.plot(t, y)
plt.ylabel('Normal')

# differencing
plt.figure(figsize=(9,3))
plt.plot(t, y - y.shift(12), 'r')
plt.ylabel('12 Month Differenced')

# log transform
log_y = y.apply(np.log)
plt.figure(figsize=(9,3))
plt.plot(t, log_y, 'g');
plt.ylabel('Log Transformed')

# both
plt.figure(figsize=(9,3))
plt.plot(t, log_y - log_y.shift(12), 'm')
plt.ylabel('Differenced and Log Transformed')
